# Market data / pricing 

**Simplified example**

- We use Alpha Vanteage API : [link](https://www.alphavantage.co/support/#api-key)
- Please create another key because it's limited in use, if anyone uses the same key => some will wait

**Packages needed**
- [alpha_vantage](https://github.com/RomelTorres/alpha_vantage) (pip)
- [pandas_datareader](https://pandas-datareader.readthedocs.io/en/latest/remote_data.html#fred) (pip)


## Stock quotes

In [36]:
from alpha_vantage.timeseries import TimeSeries

my_key = "QKFAWIESWQNEPHW7"

ts = TimeSeries(key=my_key,output_format='pandas', indexing_type='date')

data, meta_data = ts.get_intraday('GOOGL', outputsize='full')

Intraday quotes

In [47]:
data.tail(5)

,1. open,2. high,3. low,4. close,5. volume
date,,,,,
2019-02-04 10:45:00,1130.4301,1134.4900,1130.4301,1134.4900,81481.0
2019-02-04 11:00:00,1134.6100,1136.5699,1134.3800,1135.8000,76201.0
2019-02-04 11:15:00,1136.0050,1140.0000,1135.7100,1139.1801,96084.0
2019-02-04 11:30:00,1138.4100,1140.9399,1136.9100,1136.9250,79628.0
2019-02-04 11:45:00,1137.6799,1137.6974,1133.5901,1133.5901,55582.0


Last date

In [48]:
data.index[-1]

'2019-02-04 11:45:00'

Get the "close" at the latest date-time

In [50]:
spot = data.loc[data.index[-1]:]["4. close"].values[0]

spot

1133.5901

In [77]:
def get_spot(ticker):
    data, _ = ts.get_intraday(ticker)
    return data.loc[data.index[-1]:]["4. close"].values[0]

In [81]:
get_spot("AAPL")

171.07

## Fed funds

In [51]:
from pandas_datareader import data as pdr


fedfunds = pdr.FredReader("FEDFUNDS", start, end).read().sort_index()
fedfunds /= 100

In [52]:
fedfunds

,FEDFUNDS
DATE,
2018-02-01,0.0142
2018-03-01,0.0151
2018-04-01,0.0169
2018-05-01,0.0170
2018-06-01,0.0182
2018-07-01,0.0191
2018-08-01,0.0191
2018-09-01,0.0195
2018-10-01,0.0219


Last observed date

In [53]:
fedfunds.index[-1]

Timestamp('2019-01-01 00:00:00')

In [57]:
usd_rate = fedfunds.loc[fedfunds.index[-1], :].values[0]

usd_rate

0.024

## Computation

We will compute a forward $K$ : 
$$
K = S_0.e^{r_{USD}.T}
$$

[reference](https://math.stackexchange.com/questions/1777111/forward-price-in-black-scholes-model)

In [82]:
import math
import pandas


def bs_forward(s_0, r, t):
    return s_0 * math.exp(r * t)

In [83]:
T = 2.5  # years

bs_forward(spot, usd_rate, T)

1203.6873969820087

In [93]:
def price_usd_forward(nominal, ticker, maturity):
    
    time_to_mat = (pandas.Timestamp(maturity) -  pandas.Timestamp.now()).days / 365.0
    
    spot = get_spot(ticker)
    
    num_shares = nominal / spot
    
    return num_shares *  bs_forward(spot, usd_rate, time_to_mat)

In [95]:
p = price_usd_forward(1_000_000, "AAPL", "2020-01-03")

p

1022070.1578072651